We start by importing all libraries as follows:

In [2]:
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.utils import to_categorical

Define some of the hyperparameters:

In [3]:
n_classes = 5
batch_size = 128
n_epochs = 50

Load dataset and pre-process the training and validation data:

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   horizontal_flip=True,
                                   vertical_flip=False,
                                   validation_split=0.25)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('data',
                                                target_size = (150,150),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 subset = "training")

validation_set = train_datagen.flow_from_directory('data',
                                            target_size = (150,150),
                                            batch_size = batch_size,
                                            class_mode = 'categorical',
                                            subset = "validation")

Found 3243 images belonging to 5 classes.
Found 1080 images belonging to 5 classes.


Next, we define the the network architecture:

In [6]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=(150, 150,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 150, 150, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 150, 150, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 148, 148, 32)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 74, 74, 64)        18496     
__________

We will be using the Adam optimizer with default settings:

In [7]:
opt = Adam()
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

Next, we set our callback functions. We use early stopping to prevent overfitting, ModelCheckpoint to save our best model automatically, and TensorBoard for an analysis of our results:
<br><br>
In  callbacks, we will add **TensorBoard** as follows:

In [8]:
callbacks = [EarlyStopping(monitor='val_acc', patience=5, verbose=2), 
             ModelCheckpoint('checkpoints/weights.{epoch:02d}.hdf5', save_best_only=True), 
             TensorBoard('~/notebooks/logs', write_graph=True, write_grads=True, write_images=True,
                         embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)]

/home/sara/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/sara/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Now, we can start the tensorBoard connection. Open a terminal with below command:

## tensorboard --logdir='~/notebooks/logs' ##

Now, back in the Python environment, we can start training our model:

In [ ]:
history = model.fit_generator(
            training_set,
            epochs = n_epochs,
            verbose = 1,
            validation_data = validation_set)

Epoch 1/50
26/26 [==============================] - 361s 14s/step - loss: 2.1645 - acc: 0.2503 - val_loss: 1.5145 - val_acc: 0.3065
Epoch 2/50
 2/26 [=>............................] - ETA: 4:07 - loss: 1.5266 - acc: 0.2852

After starting to train our model, we should see data coming into our TensorBoard dashboard. 